# Network randomization

Here, permutation testing is used to asess significant connectivity between LSNs. Association between two LSNs is considered significant when mean connection strenght between them is higher that 95% values from null distribution. 

Null distribution of between LSN connectivity is calculated using `n_nulls` random networks. Each random network have same total strenght and degree distribution (but not strenght distribution) as original network. Algorithm to rewire network edges `randmio_und` comes from BCT. Each edge is rewired approximately `n_rewirings` times. 

In [1]:
import json
import os
import pathlib
from os.path import join

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from bct.algorithms.reference import randmio_und_signed
from bct.utils import BCTParamError
from dn_utils.plotting import plot_matrix
from dn_utils.path import path

%matplotlib inline

/home/kmb/Desktop/Neuroscience/Projects/BONNA_decide_net/code/dn_utils/glm_utils.py:13: FutureWarning: 

 | Starting with Nilearn 0.7.0, all Nistats functionality has been incorporated into Nilearn's stats & reporting modules.
 | Nistats package will no longer be updated or maintained.

  from nistats import design_matrix


In [2]:
path_root = os.environ.get("DECIDENET_PATH")

path_derivatives = join(path_root, "data/main_fmri_study/derivatives")
path_sourcedata = join(path_root, "data/main_fmri_study/sourcedata") 

path_beh = join(path_sourcedata, "behavioral")
path_bsc = join(path_derivatives, "bsc")
path_nistats = join(path_derivatives, "nistats")
path_parcellations = join(path_derivatives, "parcellations")

path_corrmats = join(path_bsc, "corrmats")

In [3]:
# Input networks
atlas = "combined_roi"

# Randomization options
n_nulls = 50
n_rewirings = 4

# Create output paths
path_out = join(path_corrmats, atlas, "unthr")
path_nulls = join(path_out, "nulls")
pathlib.Path(path_nulls).mkdir(exist_ok=True)

In [4]:
# Load correlation matrices and metadata
corrmats_aggregated = np.load(join(path_corrmats, atlas, 
                                   "corrmats_aggregated.npy"))
with open(join(path_corrmats, atlas, "corrmats_aggregated.json"), "r") as f:
    meta = json.loads(f.read())

# Load subject exclusion
df_exclusion = pd.read_csv(join(path_nistats, "exclusion/exclusion.csv"), 
                           index_col=0)
ok_index = df_exclusion["ok_all"]    
    
# Load ROI information
df_roi = pd.read_csv(join(path_corrmats, atlas, "roi_table_filtered.csv"))
netnames = df_roi["netName"].unique()

n_subjects = len(meta["dim1"])
n_conditions = len(meta["dim2"])
n_perr_sign = len(meta["dim3"])
n_rois = len(df_roi)
n_nets = len(netnames)

In [5]:
netmats_null = np.zeros(
    (n_subjects, n_conditions, n_perr_sign, n_nulls, n_nets, n_nets))

for sub_idx, sub in enumerate(meta["dim1"]): 
    print(f"Subject: {sub}")
    
    for con_idx, con in enumerate(meta["dim2"]):
        for perr_sign_idx, perr_sign in enumerate(meta["dim3"]):

            # Extract correlation matrix & clean diagonal
            corrmat = corrmats_aggregated[sub_idx, con_idx, perr_sign_idx]
            corrmat[np.diag_indices_from(corrmat)] = 0

            for rep in tqdm(range(n_nulls)):

                # Randomize network
                try:
                    corrmat_null, _  = randmio_und_signed(corrmat, n_rewirings)
                except BCTParamError:
                    # In case of non-symmetrical matrix from failed acquisition
                    corrmat_null = np.zeros((n_rois, n_rois))

                # Calculate lsn mean connectivity & store
                _, netmat_null = networks_mean(corrmat_null, df_roi["netName"])
            
                netmats_null[sub_idx, con_idx, perr_sign_idx, rep, :, :] = \
                    netmat_null

np.save(join(path_nulls, "netmats_null_001.npy"), netmats_null)

Subject: m02


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 